# Selected Recent Earthquakes

The [U.S. Geological Survey](https://www.usgs.gov) provides real-time earthquake data in [CSV format](https://earthquake.usgs.gov/earthquakes/feed/v1.0/csv.php).

{search_query : Query ? Filter by string, e.g. Fiji}

{sensor_network_table : Sensor Networks}

In [ ]:
target_folder = '/tmp'
search_query = ''
sensor_network_table_path = 'datasets/World-AdvancedNationalSeismicSystemNetworks.csv'

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from os.path import join
from pandas import read_csv

url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_month.csv'

earthquakes = read_csv(url, usecols=[
    'time', 'latitude', 'longitude', 'depth', 'mag', 'net', 'place',
], parse_dates=['time'])

earthquake_count = len(earthquakes)

In [ ]:
# Filter by search_query
earthquakes = earthquakes[earthquakes['place'].str.contains(search_query, case=False)]
earthquakes.ix[earthquakes.index[0]]

In [ ]:
target_path = join(target_folder, 'selected-earthquakes.csv')
sorted_earthquakes = earthquakes.sort_values('mag', ascending=False)
sorted_earthquakes.to_csv(target_path, index=False)
sorted_earthquake_count = len(sorted_earthquakes)
print('selected_earthquake_table_path = %s' % target_path)
print('selected_earthquake_count = %s' % sorted_earthquake_count)
print('selected_earthquake_percent = %s' % (100 * sorted_earthquake_count / float(earthquake_count)))
sorted_earthquakes.head()

## How were the earthquakes distributed?

In [ ]:
earthquakes.hist();

In [ ]:
target_path = join(target_folder, 'histograms.png')
plt.savefig(target_path)
print('histograms_image_path = %s' % target_path)
plt.figure()

## What was the magnitude of the largest earthquake?

In [ ]:
earthquakes['mag'].max()

## Which place had the largest earthquake?

In [ ]:
earthquakes.ix[earthquakes['mag'].idxmax()]['place']

## Plot earthquake magnitudes

In [ ]:
# Plot by day
earthquakes_by_time = earthquakes.set_index('time')
earthquakes_by_time['mag'].plot();

In [ ]:
# Plot by week
earthquakes_by_time.resample('W')['mag'].plot();

In [ ]:
target_path = join(target_folder, 'magnitudes.png')
plt.savefig(target_path)
print('magnitudes_image_path = %s' % target_path)
plt.figure()

## How many earthquakes were north of the equator?

In [ ]:
len(earthquakes[earthquakes['latitude'] > 0])

## How many earthquakes were south of the equator?

In [ ]:
# Type your solution here and press CTRL-ENTER

## How many earthquakes did each sensor network measure?

In [ ]:
# Count earthquakes by sensor network code
earthquakes['net'].value_counts()

## What were the names of each sensor network?

In [ ]:
# Load information on sensor networks
sensor_networks = read_csv(sensor_network_table_path)
sensor_networks.head()

In [ ]:
# Rename columns
sensor_networks.rename(columns={
    'Network Code': 'net',
    'Network Name': 'netName',
}, inplace=True)
# Change codes to lowercase
sensor_networks['net'] = sensor_networks['net'].str.lower()
sensor_networks.head()

In [ ]:
# Merge tables
merged_earthquakes = earthquakes.merge(sensor_networks)
merged_earthquakes[['place', 'mag', 'net', 'netName']].tail()

In [ ]:
# Count earthquakes by sensor network name
merged_earthquakes.groupby('netName')['mag'].count()

## Which ten places had the most earthquakes?

In [ ]:
earthquakes['place'].value_counts()[:10]

## Which ten places had the largest earthquakes?

In [ ]:
earthquakes.groupby('place')['mag'].max().sort_values(ascending=False)[:10]

## Where are these earthquakes on a map?

In [ ]:
from datetime import datetime
now = datetime.utcnow()

def get_elapsed_time_in_negative_seconds(x):
    return -1 * (now - x).seconds

geotable = earthquakes.copy()
geotable['fillReds'] = geotable['time'].apply(get_elapsed_time_in_negative_seconds)
geotable['radiusInPixelsRange3-27'] = geotable['mag']

target_path = join(target_folder, 'mapped-earthquakes.csv')
geotable.to_csv(target_path, index=False)
print('selected_earthquakes_geotable_path = %s' % target_path)

Information provided courtesy of the [USGS Earthquake Hazards Program](https://earthquake.usgs.gov/earthquakes/feed).

{selected_earthquake_count : selected earthquake count (past 30 days) ? # of earthquakes matching your query}